In [2]:
%pip install math_verify

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [math_verify] [math_verify]

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os
import re
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
from math_verify import verify, parse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-07-24 16:20:35.510210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-24 16:20:40.122003: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

In [3]:
def extract_boxed_answer(text: str) -> str:
    boxed_pattern = r"\\boxed\{((?:[^{}]|\{[^{}]*\})*)\}"
    matches = re.findall(boxed_pattern, text)
    
    if matches:
        return matches[-1]
    
    return ""

In [4]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""
MODEL_PATH = '/home/jupyter/datasphere/project/check_sft'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
DEVICE

'cuda'

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
model = AutoModelForCausalLM.from_pretrained("./check_sft")

In [8]:
model = model.to(DEVICE)

In [11]:
test_dataset = load_dataset("nlile/hendrycks-MATH-benchmark", split="test")
sample = test_dataset.select(range(5))

correct_answers = 0
total_answers = 0

decoded = {'problem': [], 'reasoning': [], 'answer': [], 'true_answer': []}

for example in tqdm(sample, desc="Оценка Success Rate с чекпоинта"):
    problem_text = example["problem"]
    ground_truth_answer = example["answer"]
    
    decoded['true_answer'].append(ground_truth_answer)    
    decoded['problem'].append(problem_text)

    if ground_truth_answer is None:
        continue
    total_answers += 1

    prompt = SYSTEM_PROMPT + " Problem: " + problem_text
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.01,
        top_p=1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    decoded['reasoning'].append(generated_output)
    model_answer_str = extract_boxed_answer(generated_output)
    decoded['answer'].append(model_answer_str)

    if verify(parse(model_answer_str), parse(ground_truth_answer), float_rounding=2):
        correct_answers += 1

if total_answers > 0:
    success_rate = (correct_answers / total_answers) * 100
    print("\n" + "="*50)
    print("="*50)
    print(f"✅ Верных ответов: {correct_answers} из {total_answers}")
    print(f"📈 Итоговый Success Rate: {success_rate:.2f}%")
    print("="*50)
else:
    print("Не найдено примеров с эталонным ответом для оценки.")

Оценка Success Rate с чекпоинта: 100%|██████████| 5/5 [01:22<00:00, 16.59s/it]


✅ Верных ответов: 3 из 5
📈 Итоговый Success Rate: 60.00%


In [12]:
decoded

{'problem': ['Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$',
  'Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$',
  'If $f(x) = \\frac{3x-2}{x-2}$, what is the value of $f(-2) +f(-1)+f(0)$? Express your answer as a common fraction.',
  'How many positive whole-number divisors does 196 have?',
  'The results of a cross-country team\'s training run are graphed below. Which student has the greatest average speed? [asy]\nfor ( int i = 1; i <= 7; ++i )\n{\n\ndraw((i,0)--(i,6));\n}\n\nfor ( int i = 1; i <= 5; ++i )\n{\n\ndraw((0,i)--(8,i));\n}\ndraw((-0.5,0)--(8,0), linewidth(1));\ndraw((0,-0.5)--(0,6), linewidth(1));\nlabel("$O$", (0,0), SW);\nlabel(scale(.85)*rotate(90)*"distance", (0